In [6]:
#Удаление комментариев
def delete_com(filename, path_in, path_out):
    heh = set({'+', '-', '*', '/', '.', ',', '(', ')', '{', '}', '&', '%', '@', '!', '=', '>', '<', ':', '~', '[', ']', '|'})
    with open(path_out + '/' + filename, 'w', encoding='utf-8') as outfile:
        f = True
        with open(path_in + '/' + filename, 'r', encoding='utf-8') as infile:
            c = infile.read(1)
            while c:
                if c == '/':
                    c = infile.read(1)
                    if c == '*':
                        f = False
                    elif c == '/':
                        infile.readline()
                        outfile.write('\n')
                    elif f:
                        outfile.write('/')
                        if c in heh:
                            outfile.write(' ')
                            outfile.write(c)
                            outfile.write(' ')
                        else:
                            outfile.write(c)
                elif c == '*':
                    c = infile.read(1)
                    if c == '/':
                        f = True
                    elif f:
                        outfile.write('*')
                        if c in heh:
                            outfile.write(' ')
                            outfile.write(c)
                            outfile.write(' ')
                        else:
                            outfile.write(c)
                elif f and c != ';':
                    if c in heh:
                        outfile.write(' ')
                        outfile.write(c)
                        outfile.write(' ')
                    else:
                        outfile.write(c)
                c = infile.read(1)

In [8]:
import os
path_in = 'True_program'
path_out = 'True_program_without_com'

for filename in os.listdir(path_in):
    delete_com(filename, path_in, path_out)

In [18]:
#Формирования словаря слов для всех файлов 
def load_dir(filename, path):
    global d
    global CNT_COL
    cnt = 0
    with open(path + '/' + filename, 'r', encoding='utf-8') as infile:
        for line in infile:
            s = line.split()
            for i in s:
                if i in d.keys():
                    d[i] += 1
                else:
                    d[i] = 1
                cnt += 1
    if cnt > CNT_COL:
        CNT_COL = cnt
            


In [19]:
import numpy as np
import os

CNT_ROW_TRUE = 0
CNT_COL = 0

d = {}

path = 'True_program_without_com'

for filename in os.listdir(path):
    load_dir(filename, path)
    CNT_ROW_TRUE += 1

In [20]:
CNT_ROW_FALSE = 0

path = 'False_program_without_com'

for filename in os.listdir(path):
    load_dir(filename, path)
    CNT_ROW_FALSE += 1

In [21]:
data_true = np.zeros((CNT_ROW_TRUE, CNT_COL))
data_false = np.zeros((CNT_ROW_FALSE, CNT_COL))

In [22]:
l = []
for i in d.keys():
    l += [[d[i],i]]
l.sort()

In [23]:
my_dir = {}
for i in range(len(l)):
    my_dir[l[i][1]] = len(l) - i

In [24]:
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import imdb

Using TensorFlow backend.


In [25]:
j = 0

path = 'True_program_without_com'
for filename in os.listdir(path):
    with open(path + '/' + filename, 'r', encoding='utf-8') as infile:
        for line in infile:
            s = line.split()
            for i in range(len(s)):
                data_true[j][i] = my_dir[s[i]]
    j += 1
    

In [26]:
j = 0

path = 'False_program_without_com'
for filename in os.listdir(path):
    with open(path + '/' + filename, 'r', encoding='utf-8') as infile:
        for line in infile:
            s = line.split()
            for i in range(len(s)):
                data_false[j][i] = my_dir[s[i]]
    j += 1
    

In [ ]:
target_true = np.ones((CNT_ROW_TRUE, 1))
target_false = np.zeros((CNT_ROW_FALSE, 1))

In [ ]:
data = np.concatenate((data_true, data_false), axis=0)
target = np.concatenate((target_true, target_false), axis=0)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 11)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
import time

In [ ]:
#RNN

from keras.layers import LSTM
from keras.layers import Dense

model = models.Sequential()
# Input - Layer
model.add(LSTM(32))
#model.add(Dense(50, activation = "relu"))
model.add(Dense(1, activation='sigmoid'))
#model.summary()
# compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

time_start = time.time()

results = model.fit(
    X_train, y_train,
    batch_size=50,
    epochs=3,
    verbose=1,
    validation_data = (X_test, y_test),
)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))
print("Time: ", time.time() - time_start)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 2672 samples, validate on 669 samples
Epoch 1/3
 250/2672 [=>............................] - ETA: 17:54 - loss: 0.6924 - acc: 0.5440

In [26]:
from keras.callbacks import TensorBoard  
tensorboard=TensorBoard(log_dir='./logs', write_graph=True)

results = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=3,
    verbose=1,
    validation_data = (X_test, y_test),
    callbacks=[tensorboard]
)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))

Train on 2672 samples, validate on 669 samples
Epoch 1/3
2672/2672 [==============================] - 402s 151ms/step - loss: 0.6883 - acc: 0.5520 - val_loss: 0.6901 - val_acc: 0.5396
Epoch 2/3
2672/2672 [==============================] - 365s 137ms/step - loss: 0.6878 - acc: 0.5520 - val_loss: 0.6901 - val_acc: 0.5396
Epoch 3/3
2672/2672 [==============================] - 367s 138ms/step - loss: 0.6878 - acc: 0.5520 - val_loss: 0.6903 - val_acc: 0.5396
Test-Accuracy: 0.5396113641593488


In [42]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

In [41]:
model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(X_train.shape[1], )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()
# compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)
results = model.fit(
 X_train, y_train,
 epochs= 50,
 batch_size = 30,
 validation_data = (X_test, y_test)
)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 50)                1027400   
_________________________________________________________________
dropout_21 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_22 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 51        
Total params: 1,032,551
Trainable params: 1,032,551
Non-trainable params: 0
_________________________________________________________________


In [52]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)
print("Train-Accuracy:", knn.score(X_train, y_train))
print("Test-Accuracy:", knn.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


Train-Accuracy: 0.7095808383233533
Test-Accuracy: 0.5844544095665172


In [53]:
time_start = time.time()
y_pred = knn.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  103.60171341896057


In [65]:
# Логистическая регрессия
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C = 0.001, penalty='l1')
LR.fit(X_train, y_train)
print("Train-Accuracy:", LR.score(X_train, y_train))
print("Test-Accuracy:", LR.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train-Accuracy: 0.6347305389221557
Test-Accuracy: 0.6023916292974589


In [66]:
time_start = time.time()
y_pred = LR.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.05309581756591797


In [74]:
# Метод опорных векторов
from sklearn.svm import LinearSVC
LSVC = LinearSVC()
LSVC.fit(X_train, y_train)
print("Train-Accuracy:", LSVC.score(X_train, y_train))
print("Test-Accuracy:", LSVC.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train-Accuracy: 0.5520209580838323
Test-Accuracy: 0.5142002989536621


In [75]:
time_start = time.time()
y_pred = LSVC.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.06216597557067871


In [91]:
# Деревья решения
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)
print("Train-Accuracy:", tree.score(X_train, y_train))
print("Test-Accuracy:", tree.score(X_test, y_test))

Train-Accuracy: 0.9966317365269461
Test-Accuracy: 0.6890881913303438


In [92]:
time_start = time.time()
y_pred = tree.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.1286630630493164


In [101]:
# Случайный лес
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=500, random_state=0)
forest.fit(X_train, y_train)
print("Train-Accuracy:", forest.score(X_train, y_train))
print("Test-Accuracy:", forest.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Train-Accuracy: 0.9966317365269461
Test-Accuracy: 0.7757847533632287


In [102]:
time_start = time.time()
y_pred = forest.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.21543002128601074


In [111]:
# Градиентный бустинг
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(random_state=0, n_estimators=100)
gbrt.fit(X_train, y_train)
print("Train-Accuracy:", gbrt.score(X_train, y_train))
print("Test-Accuracy:", gbrt.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train-Accuracy: 0.8278443113772455
Test-Accuracy: 0.7279521674140508


In [112]:
time_start = time.time()
y_pred = gbrt.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.12330150604248047


In [116]:
# Нейроночка
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[10, 10])
mlp.fit(X_train, y_train)
print("Train-Accuracy:", mlp.score(X_train, y_train))
print("Test-Accuracy:", mlp.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train-Accuracy: 0.5426646706586826
Test-Accuracy: 0.4962630792227205


In [117]:
time_start = time.time()
y_pred = mlp.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.10277390480041504


In [119]:
# Байес
from sklearn.naive_bayes import GaussianNB
gNB = GaussianNB()
gNB.fit(X_train, y_train)
print("Train-Accuracy:", gNB.score(X_train, y_train))
print("Test-Accuracy:", gNB.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train-Accuracy: 0.4498502994011976
Test-Accuracy: 0.46038863976083705


In [120]:
time_start = time.time()
y_pred = gNB.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  1.0944085121154785


In [1]:
import gensim 
from gensim.models import Word2Vec

C:\Users\Polya\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\Polya\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
word_data = []
import os

path = 'True_program_without_com'
for filename in os.listdir(path):
    with open(path + '/' + filename, 'r', encoding='utf-8') as infile:
        for line in infile:
            s = line.split()
            word_data += [s]
            
path = 'False_program_without_com'
for filename in os.listdir(path):
    with open(path + '/' + filename, 'r', encoding='utf-8') as infile:
        for line in infile:
            s = line.split()
            word_data += [s]

word_model = gensim.models.Word2Vec(word_data, min_count = 1, size = 5, window = 10)

In [9]:
import numpy as np
data_true = np.zeros((CNT_ROW_TRUE, CNT_COL*5))
data_false = np.zeros((CNT_ROW_FALSE, CNT_COL*5))

In [10]:

j = 0

path = 'True_program_without_com'
for filename in os.listdir(path):
    with open(path + '/' + filename, 'r', encoding='utf-8') as infile:
        for line in infile:
            s = line.split()
            for i in range(len(s)):
                data_true[j][i*5: i*5+5] = word_model[s[i]]
    j += 1
    

C:\Users\Polya\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [11]:
j = 0

path = 'False_program_without_com'
for filename in os.listdir(path):
    with open(path + '/' + filename, 'r', encoding='utf-8') as infile:
        for line in infile:
            s = line.split()
            for i in range(len(s)):
                data_false[j][i*5: i*5+5] = word_model[s[i]]
    j += 1
    

C:\Users\Polya\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [12]:
target_true = np.ones((CNT_ROW_TRUE, 1))
target_false = np.zeros((CNT_ROW_FALSE, 1))

In [13]:
data = np.concatenate((data_true, data_false), axis=0)
target = np.concatenate((target_true, target_false), axis=0)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 11)

In [180]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [181]:
#RNN

from keras.layers import LSTM
from keras.layers import Dense

model = models.Sequential()
# Input - Layer
model.add(LSTM(32))
#model.add(Dense(50, activation = "relu"))
model.add(Dense(1, activation='sigmoid'))
#model.summary()
# compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

time_start = time.time()
results = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=3,
    verbose=1,
    validation_data = (X_test, y_test),
)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))
print("Time: ", time.time() - time_start)

Train on 2672 samples, validate on 669 samples
Epoch 1/3
2672/2672 [==============================] - 6153s 2s/step - loss: 0.6899 - acc: 0.5460 - val_loss: 0.6902 - val_acc: 0.5396
Epoch 2/3
2672/2672 [==============================] - 4081s 2s/step - loss: 0.6877 - acc: 0.5520 - val_loss: 0.6907 - val_acc: 0.5396
Epoch 3/3
2672/2672 [==============================] - 5189s 2s/step - loss: 0.6880 - acc: 0.5520 - val_loss: 0.6903 - val_acc: 0.5396
Test-Accuracy: 0.5396113641593488
Time:  15444.473833560944


In [15]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

NameError: name 'X_train' is not defined

In [100]:
model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(X_train.shape[1], )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()
# compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)
results = model.fit(
 X_train, y_train,
 epochs= 50,
 batch_size = 30,
 validation_data = (X_test, y_test)
)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 50)                1027400   
_________________________________________________________________
dropout_23 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_24 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 51        
Total params: 1,032,551
Trainable params: 1,032,551
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print("Train-Accuracy:", knn.score(X_train, y_train))
print("Test-Accuracy:", knn.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [135]:
time_start = time.time()
y_pred = knn.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  111.93881106376648


In [148]:
# Логистическая регрессия
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)
print("Train-Accuracy:", LR.score(X_train, y_train))
print("Test-Accuracy:", LR.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train-Accuracy: 0.6302395209580839
Test-Accuracy: 0.6128550074738416


In [147]:
time_start = time.time()
y_pred = LR.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.07570171356201172


In [152]:
from sklearn.svm import LinearSVC
LSVC = LinearSVC()
LSVC.fit(X_train, y_train)
print("Train-Accuracy:", LSVC.score(X_train, y_train))
print("Test-Accuracy:", LSVC.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train-Accuracy: 0.5684880239520959
Test-Accuracy: 0.5575485799701047


In [153]:
time_start = time.time()
y_pred = LSVC.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.0661768913269043


In [17]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)
print("Train-Accuracy:", tree.score(X_train, y_train))
print("Test-Accuracy:", tree.score(X_test, y_test))

Train-Accuracy: 0.9966317365269461
Test-Accuracy: 0.6846038863976084


In [159]:
time_start = time.time()
y_pred = tree.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.14388155937194824


In [166]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=500, random_state=0)
forest.fit(X_train, y_train)
print("Train-Accuracy:", forest.score(X_train, y_train))
print("Test-Accuracy:", forest.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Train-Accuracy: 0.9962574850299402
Test-Accuracy: 0.7623318385650224


In [167]:
time_start = time.time()
y_pred = forest.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.3283720016479492


In [172]:
# Градиентный бустинг
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(random_state=0, n_estimators=100, learning_rate = 1)
gbrt.fit(X_train, y_train)
print("Train-Accuracy:", gbrt.score(X_train, y_train))
print("Test-Accuracy:", gbrt.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train-Accuracy: 0.9820359281437125
Test-Accuracy: 0.7563527653213752


In [173]:
time_start = time.time()
y_pred = gbrt.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.17696928977966309


In [176]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes = [10, 10])
mlp.fit(X_train, y_train)
print("Train-Accuracy:", mlp.score(X_train, y_train))
print("Test-Accuracy:", mlp.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train-Accuracy: 0.5033682634730539
Test-Accuracy: 0.49327354260089684


In [177]:
time_start = time.time()
y_pred = mlp.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  0.10377621650695801


In [178]:
from sklearn.naive_bayes import GaussianNB
gNB = GaussianNB()
gNB.fit(X_train, y_train)
print("Train-Accuracy:", gNB.score(X_train, y_train))
print("Test-Accuracy:", gNB.score(X_test, y_test))

C:\Users\Polya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train-Accuracy: 0.4498502994011976
Test-Accuracy: 0.46038863976083705


In [179]:
time_start = time.time()
y_pred = gNB.predict(X_test)
print("Time: ", time.time() - time_start)

Time:  2.4997010231018066
